$p_T[n] = \frac{(rt)^n}{n!}e^{-rT}$

* https://www.tu-chemnitz.de/informatik/KI/scripts/ws0910/Neuron_Poisson.pdf
* http://www.cns.nyu.edu/~david/handouts/poisson.pdf

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets


def poisson_generator(r=25, T=1000, N=100, dt=0.1):
    """This scheme models a homogeneous Poisson process by 
    discretising time and is appropriate for when dt << 1.
    """

    # Binning methods: 'auto', 'sturges', 'fd', 'doane', 'scott', 'rice', 'sturges' 'sqrt'
    bins = 'fd'
    
    r /= 1000  # Rescale because time is in ms
    spikes = np.random.rand(N, int(T/dt)) < (r*dt)
#     indices = np.argwhere(spikes)
#     n_ind, t_ind = indices[:, 0], indices[:, 1]
    # Equivalently
    n_ind, t_ind = np.where(spikes)

    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18,6))
    axes[0].plot(t_ind*dt, 1+n_ind, '|')
    axes[0].set_xlim(0, T)
    axes[0].set_ylim(0, N+1)
    axes[0].set_xlabel("Time [ms]")
    axes[0].set_ylabel("$i$")

    # Gaussian distribution
    counts = np.sum(spikes, axis=1)
    rates = counts*T/1000
    # On average mean = variance in a Poisson process, so fano = 1
    r_var, r_mean = np.var(counts), np.mean(counts)
    fano = r_var / r_mean
    # print(f"Fano factor = {fano:.3}")
    axes[1].hist(rates, bins=bins, label=f"$\mu$={r_mean:.3}\n$\sigma^2$={r_var:.3}\n$F$={fano:.3}")
    axes[1].set_xlabel("Spike rate [Hz]")
    axes[1].set_ylabel("$f$")
    axes[1].legend()
    
    ISIs = [np.diff(t_ind[np.where(n_ind == i)] * dt) for i in range(N)]
    ISIs = np.concatenate(ISIs).ravel()
    isi_std, isi_mean = np.std(ISIs), np.mean(ISIs)
    cv = isi_std / isi_mean
    # print(f"Coefficient of variation = {cv:.3}")
    # Exponential distribution
    axes[2].hist(ISIs, bins=bins, label=f"$\mu$={isi_mean:.3}\n$\sigma$={isi_std:.3}\n$C_V$={cv:.3}")
    axes[2].set_xlabel("ISI [ms]")
    axes[2].set_ylabel("$f$")
    axes[2].legend()


interact(poisson_generator, r=(0,100.), T=(10,1000), N=(1,1000), dt=widgets.FloatLogSlider(value=0.1, base=10, min=-3, max=0))